<a href="https://colab.research.google.com/github/sabrisangjaya/sabrisangjaya.github.io/blob/master/Skripsi/Hasil/Ekstrasi_Fitur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import re
import sys
from nltk.tokenize import word_tokenize,sent_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import pandas as pd
import numpy as np
import PyPDF2
from collections import Counter
import json
import ast

data=[]

khayalan2= np.loadtxt('khayalan new.txt', delimiter='\n',dtype='str')
realistis2= np.loadtxt('realistis new.txt', delimiter='\n',dtype='str')
romantis2= np.loadtxt('romantis new.txt', delimiter='\n',dtype='str')
khayalan2,realistis2,romantis2=[x.lower() for x in khayalan2],[x.lower() for x in realistis2],[x.lower() for x in romantis2]
khayalan2=sorted(list(set(khayalan2)))
realistis2=sorted(list(set(realistis2)))
romantis2=sorted(list(set(romantis2)))

factory2 = StopWordRemoverFactory()
stopwords = factory2.get_stop_words()
stopwordbahasa= np.loadtxt('stopwordbahasa.txt', delimiter='\n',dtype='str')
stopwordbahasa=[x.lower() for x in stopwordbahasa]
stopwords=stopwords+stopwordbahasa
stopwords=sorted(list(set(stopwords)))

rootdir ='txt/coba'
import traceback
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        try:
            njpg = 0
            strall=open(os.path.join(subdir, file)).read()
            fileoutput="before"+file[-3:]+" "+file[:-4]+".txt"
            with open(os.path.join('txt/outputtxt/',fileoutput),"w",encoding='UTF-8') as text_file:
                text_file.write(strall)
            try:
                filepdf = open(os.path.join('txt/pdf/', file[:-4]+".pdf"), 'rb')
                pdfReader = PyPDF2.PdfFileReader(filepdf)
                jumlahhalaman=pdfReader.getNumPages()
            except:
                jumlahhalaman=0
                print("PDF not found")
            #print(strall)
            #preprocessing
            strall=strall.lower()
            strall=re.sub(r'\d+', '',strall) #Menghapus Angka
            nlines = strall.count('\n')
            strall=re.sub(r'\n', ' ', strall) #Mengganti encoding enter
            strall=re.sub(r'	', ' ', strall) #Mengganti encoding tab
            strall=re.sub(' +', ' ',strall) #Menghapus spasi ganda
            strall=re.sub('\.\.\.', '…',strall) #Mengganti titik tiga agar tidak split kalimat
            fileoutput="after"+file[-3:]+" "+file[:-4]+".txt"
            with open(os.path.join('txt/outputtxt/',fileoutput),"w") as text_file:
                text_file.write(strall)
            #print(fileoutput)
            #print(strall)
            #Generate Atribut
            kalimat=sent_tokenize(strall) #tokenize kalimat
            kalimat=[x for x in kalimat if len(x)!=1]
            totalkata=word_tokenize(strall) #tokenize kata
            totalkata=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]', '', x), totalkata))) #filter hasil tokenize
            totalkata=list(map(str.lower,totalkata)) #lowercase tokenize
            totalkata=sorted(totalkata)
            counts = dict(Counter(totalkata)) #menghitung kemunculan kata
            counts = {k: v for k, v in sorted(counts.items(), key=lambda item: item[1],reverse=True)}
            jeniskata=list(set(totalkata)) #menghitung jenis kata
            sukukata=[] #variabel inisialisasi
            sukukatapoly=[] #variabel inisialisasi
            sukukatanonpoly=[] #variabel inisialisasi
            totalsuku=0 #variabel inisialisasi
            ro_count,re_count,kh_count,ge_count=0,0,0,0
            ro_list,re_list,kh_list,ge_list=[],[],[],[]
            a=counts
            for j in a:
                if(j in khayalan2 and (j not in stopwords)):
                    kh_count+=a[j]
                    kh_list.append([j,a[j]])
                if(j in realistis2 and (j not in stopwords)):
                    re_count+=a[j]
                    re_list.append([j,a[j]])
                if(j in romantis2 and (j not in stopwords)):
                    ro_count+=a[j]
                    ro_list.append([j,a[j]])
                if((j not in khayalan2) and (j not in realistis2) and (j not in romantis2) and (j not in stopwords)):
                    ge_count+=a[j]
                    ge_list.append([j,a[j]])
            #print(row['Nama File'],"=",ro_count,"/",re_count,"/",kh_count)
            #print("Romantis :",ro_count)
            #print(ro_list)
            #print("Realistis :",re_count)
            #print(re_list)
            #print("Khayalan :",kh_count)
            #print(kh_list)

            for i in jeniskata: #menghitung sukukata
                count = 0
                for j in i:
                    if re.match(r'^[bcdfghjklmnpqrstvwxyz]*[aeiou]+[bcdfghjklmnpqrstvwxyz]*$', j):
                        count += 1
                        totalsuku+=1
                if(count<=2):
                    sukukatanonpoly.append([i,count])
                elif(count>2):
                    sukukatapoly.append([i,count])
                sukukata.append([i,count])
            print(jumlahhalaman) #Jumlah Halaman
            print(nlines) #Jumlah Paragraf
            print(len(kalimat))#Jumlah Kalimat
            print(len(totalkata)) #Jumlah Total Kata
            print(len(jeniskata)) #Jumlah Jenis Kata
            #print(counts)#Kemunculan Kata
            print(totalsuku)#Total Suku Kata
            #print(sukukata)#Kata dan jumlah Suku kata
            print(len(sukukatanonpoly)) #jumlah kata yang suku kata kurang dari sama dengan 2
            print(len(sukukatapoly)) #jumlah kata yang suku kata lebih dari 2
            
            dataappend=[file,
                        jumlahhalaman,
                        kalimat,
                        totalkata,
                        jeniskata,
                        sukukata,
                        sukukatapoly,
                        sukukatanonpoly,
                        counts,
                        ro_list,re_list,kh_list,ge_list,
                        nlines,
                        len(kalimat),
                        len(totalkata),
                        len(jeniskata),
                        totalsuku,
                        len(sukukatanonpoly),
                        len(sukukatapoly),
                        ro_count,re_count,kh_count,ge_count
                       ]
            #print(dataappend)
            data.append(dataappend)
        except Exception as e:
            print("gagal",e,file,traceback.format_exc())
            pass
df=pd.DataFrame(data)
df.columns=["Nama File","Jumlah Halaman","kalimat","kata","jenis kata","suku kata","suku kata poly",
            "suku kata nonpoly","kemunculan kata",
            "listromantis","listrealistis","listkhayalan","listgeneral",
            "paragraf","jumlah kalimat","jumlah total kata",
            "jumlah jenis kata","total suku kata","jumlah suku kata non poly","jumlah suku kata poly",
            "romantis","realistis","khayalan","general"]
print(df)
df.to_excel("hasilgenerate2.xlsx")
#df.to_csv("hasilgenerate.csv")
print("Selesai")